#### 2-1. Nate 뉴스기사 제목 스크래핑하기
https://news.nate.com/recent?mid=n0100   
최신뉴스, 정치 , 경제, 사회, 세계, IT/과학 6개의 섹션의 뉴스를 출력하는 함수를 생성하여 스크래핑 하기


In [16]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin  # url 합치는 도구
from IPython.display import Image, display # 이미지를 한 번에 보여주는 도구

In [17]:
section_dict = {'최신뉴스': 'n0100','정치': 'n0200',
                '경제': 'n0300','사회': 'n0400',
                '세계': 'n0500','IT/과학': 'n0600'}

In [23]:
def scrap_nate_news(section_name):
    mid = section_dict.get(section_name)
    url = f"https://news.nate.com/recent?mid={mid}"
    
    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    res = requests.get(url, headers=req_header)
    res.encoding = 'euc-kr' # 네이트는 이거?

    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        print(f"======> {url} [{section_name}] 뉴스 <======")

        # <div class="mduSubjectList"> 가 하나의 기사 덩어리
        articles = soup.select("div.mduSubjectList")

        for article in articles:
            # 이미지 처리
            img_tag = article.select_one("span.ib img")
            
            if img_tag:
                img_src = img_tag['src']
                # //thumbnews... 로 시작하므로 urljoin 필수
                full_img_url = urljoin(url, img_src)
                
                print("[이미지]")
                display(Image(url=full_img_url, width=120))
            else:
                pass # 이미지 없으면 패스

            # 제목 은 h2의 tit
            title_tag = article.select_one("h2.tit")
            
            # 링크는 a 태그의 lt1
            link_tag = article.select_one("a.lt1")

            if title_tag and link_tag:
                title = title_tag.text.strip()
                link = link_tag['href']  # href 속성에 링크만을 가져오자
                full_link = urljoin(url, link)

                print(f"[제목] {title}")
                print(f"[링크] {full_link}")
                print("----------------------------------------------------")

    else:
        print(f"Error Code: {res.status_code}")

In [24]:
scrap_nate_news('정치')

======> https://news.nate.com/recent?mid=n0200 [정치] 뉴스 <======
[이미지]


[제목] 북한, 신평군·선천군 지방발전정책 대상 건설 착공식 진행
[링크] https://news.nate.com/view/20260208n17380?mid=n0200
----------------------------------------------------
[이미지]


[제목] 북한, 신평군·선천군 지방발전정책 대상 건설 착공식 진행
[링크] https://news.nate.com/view/20260208n17379?mid=n0200
----------------------------------------------------
[이미지]


[제목] 북한, 신평군·선천군 지방발전정책 대상 건설 착공식 진행
[링크] https://news.nate.com/view/20260208n17378?mid=n0200
----------------------------------------------------
[이미지]


[제목] 북한, 신평군·선천군 지방발전정책 대상 건설 착공식 진행
[링크] https://news.nate.com/view/20260208n17377?mid=n0200
----------------------------------------------------
[이미지]


...

[제목] "'김옥균 프로젝트'로 제명"…한동훈, 토크콘서트로 '세 과시'
[링크] https://news.nate.com/view/20260208n17209?mid=n0200
----------------------------------------------------
[이미지]


[제목] 지도부 "이력 몰랐다" 진화나섰지만…친명계 "이게 원팀이냐"
[링크] https://news.nate.com/view/20260208n17177?mid=n0200
----------------------------------------------------
[이미지]


[제목] 1만5천 팬 불러 '토크콘서트' 연 한동훈 "제풀에 그만둘 거란 기대 접으라"
[링크] https://news.nate.com/view/20260208n16395?mid=n0200
----------------------------------------------------
[이미지]


[제목] 조국 "13일까지 답 달라"…與 "의총 후 조속 정리"
[링크] https://news.nate.com/view/20260208n17157?mid=n0200
----------------------------------------------------
[이미지]


[제목] '제명' 한동훈 "그만둘 거란 기대 접길…좋은 정치 하겠다"
[링크] https://news.nate.com/view/20260208n17146?mid=n0200
----------------------------------------------------


#### 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기


In [17]:
import os

In [20]:

def download_one_episode(title,no,url):

    req_header = {
        'referer': url,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }
    res = requests.get(url, headers=req_header)

    if res.ok:
        # 이미지 url 추출
        soup = BeautifulSoup(res.text, 'html.parser')
        img_tags = soup.select("img[src*='IMAG01']") # img 태그의 src 속성에서 'IMG01' 포함한 모든 태그 추출


        #리스트 컴프리헨션
        img_url_list = [img_tag['src'] for img_tag in img_tags]  
        # img_url_list2 에 모든 이미지 링크가 배열됨

        dir_path = os.path.join('img', title, str(no))
        if not os.path.exists(dir_path): 
            os.makedirs(dir_path)

        for img_url in img_url_list:
            res = requests.get(img_url, headers=req_header)  #header를 설정 안하면 403(권한 없음이 뜬다)
            img_data = res.content

            # url에서 파일명만 추출하기
            # img/xxxIMG01.jpg
            file_path = os.path.join(dir_path, os.path.basename(img_url))

            # binary 데이터를 file에 write 하기
            with open(file_path,'wb') as file:
                print(f'Writing to {file_path}({len(img_data):,}bytes)')
                file.write(img_data)

    else:
        print(f'Error Code = {res.status_code}')




In [21]:
# 실행 테스트
download_one_episode('엄마를 만나러 가는 길', 1, 'https://comic.naver.com/webtoon/detail?titleId=834369&no=1&week=finish')

Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_1.jpg(31,242bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_2.jpg(73,417bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_3.jpg(101,481bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_4.jpg(81,404bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_5.jpg(172,033bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_6.jpg(111,113bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_7.jpg(50,877bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_8.jpg(172,144bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_9.jpg(96,121bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b